<a href="https://colab.research.google.com/github/mukulsn/Machine-Learning/blob/main/HuggingFace/summarisation_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Date : 18 Nov 2023
- Aim is to do news article summarisation.
- donwload the model and weights for future use
- What this notebook have
  - useful functions
  - creating subset of dataset
  -


In [1]:

!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [2]:
from datasets import load_dataset

# spanish_dataset = load_dataset("amazon_reviews_multi", "es")
# english_dataset = load_dataset("amazon_reviews_multi", "en")
english_dataset = load_dataset("cnn_dailymail",'1.0.0')

english_dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [3]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [53]:
wrapper = textwrap.TextWrapper(width=70)

In [4]:
import textwrap
def show_samples(dataset, num_samples=3, seed=42):
  sample = dataset['train'].shuffle(seed=seed).select(range(num_samples))
  wrapper = textwrap.TextWrapper(width=70) # wrapping text

  for example in sample:
    print(f"\n'>> Highlight : {wrapper.fill(example['highlights'])}")
    print(f"\n'>> Article : {wrapper.fill(example['article'])}")

show_samples(english_dataset)


'>> Highlight : John and . Audrey Cook were discovered alongside their daughter,
Maureen . They were found at Tremarle Home Park in Cornwall .
Investigators say the three died of carbon monoxide . poisoning .

'>> Article : By . Anthony Bond . PUBLISHED: . 07:03 EST, 2 March 2013 . | .
UPDATED: . 08:07 EST, 2 March 2013 . Three members of the same family
who died in a static caravan from carbon monoxide poisoning would have
been unconscious 'within minutes', investigators said today. The
bodies of married couple John and Audrey Cook were discovered
alongside their daughter, Maureen, at the mobile home they shared on
Tremarle Home Park in Camborne, west Cornwall. The inquests have now
opened into the deaths last Saturday, with investigators saying the
three died along with the family's pet dog, of carbon monoxide
poisoning from a cooker. Tragic: The inquests have opened into the
deaths of three members of the same family who were found in their
static caravan last weekend. John and Aud

In [5]:
english_dataset['train'] = english_dataset['train'].select(range(10000))
english_dataset['validation'] = english_dataset['validation'].select(range(8000))
# english_dataset['test'] = english_dataset['train'].select(range(10000))
english_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

Tokenising of dataset

In [6]:
from transformers import AutoTokenizer

model_checkpoint = 'google/mt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
inputs = tokenizer('i love reading the books')
tokenizer.convert_ids_to_tokens(inputs.input_ids) # converting back input ids to words of tokenizer

['▁', 'i', '▁love', '▁reading', '▁the', '▁books', '</s>']

To tokenize our corpus, we have to deal with a subtlety associated with summarization: because our labels are also text, it is possible that they exceed the model’s maximum context size. This means we need to apply truncation to both the reviews and their titles to ensure we don’t pass excessively long inputs to our model. The tokenizers in 🤗 Transformers provide a nifty text_target argument that allows you to tokenize the labels in parallel to the inputs. Here is an example of how the inputs and targets are processed for mT5:

In [8]:
%time
max_input_length = 512
max_target_length = 30

def preprocess_function(examples):
  model_inputs = tokenizer(
      examples['article'],
      max_length=max_input_length,
      truncation=True,
  )
  labels = tokenizer(
      examples['highlights'],
      max_length = max_target_length,
      truncation=True
  )
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [9]:
tokenized_datasets = english_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [10]:
generated_summary = 'I absolutely loved reading the Hunger Games'
reference_summary = 'T loved reading the Hunder Games'

In [11]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=2510ed02b4352d1b67629bbc2cf5bfff7eb3f5afd6ea6dbcc6eb300c4bf42cb8
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [12]:
import evaluate
rouge_score = evaluate.load('rouge')

the precision and recall numbers match up! Now what about those other ROUGE scores? rouge2 measures the overlap between bigrams (think the overlap of pairs of words), while rougeL and rougeLsum measure the longest matching sequences of words by looking for the longest common substrings in the generated and reference summaries. The “sum” in rougeLsum refers to the fact that this metric is computed over a whole summary, while rougeL is computed as the average over individual sentences.

In [13]:
scores = rouge_score.compute(
    predictions=[generated_summary],references=[reference_summary]
)
scores

{'rouge1': 0.6153846153846153,
 'rouge2': 0.3636363636363636,
 'rougeL': 0.6153846153846153,
 'rougeLsum': 0.6153846153846153}

We’ll use these ROUGE scores to track the performance of our model, but before doing that let’s do something every good NLP practitioner should do: create a strong, yet simple baseline!

###Creating a strong baseline
A common baseline for text summarization is to simply take the first three sentences of an article, often called the lead-3 baseline. We could use full stops to track the sentence boundaries, but this will fail on acronyms like “U.S.” or “U.N.” — so instead we’ll use the nltk library, which includes a better algorithm to handle these cases. You can install the package using pip as follows:

In [14]:
!pip install nltk

In [15]:
# downloading punctuations from NLTK

import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
from nltk.tokenize import sent_tokenize

def three_sentence_summary(text):
  return '\n'.join(sent_tokenize(text)[:3])

print(three_sentence_summary(english_dataset['train'][1]['article']))

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."


In [17]:
def evaluate_baseline(dataset, metric):
  summaries = [three_sentence_summary(text) for text in dataset['article']]
  return metric.compute(predictions=summaries, references=dataset['highlights'])


### Baseline model

In [18]:
import pandas as pd

score = evaluate_baseline(english_dataset['validation'],rouge_score)
rouge_names = ['rogue1','rogue2','rogueL','rougeLsum']
score

{'rouge1': 0.37963759512605183,
 'rouge2': 0.16548452396017777,
 'rougeL': 0.24115228892432994,
 'rougeLsum': 0.28387175543918497}

###Fine-tuning mT5 with Keras
Fine-tuning a model for summarization is very similar to the other tasks we’ve covered in this chapter. The first thing we need to do is load the pretrained model from the mt5-small checkpoint. Since summarization is a sequence-to-sequence task, we can load the model with the TFAutoModelForSeq2SeqLM class, which will automatically download and cache the weights:

In [19]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at google/mt5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [20]:
# total 30 crore trainable parmeters
model.summary()

Model: "tfmt5_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  128057344 
                                                                 
 encoder (TFT5MainLayer)     multiple                  146940608 
                                                                 
 decoder (TFT5MainLayer)     multiple                  153236160 
                                                                 
 lm_head (Dense)             multiple                  128057344 
                                                                 
Total params: 300176768 (1.12 GB)
Trainable params: 300176768 (1.12 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
from huggingface_hub import notebook_login

notebook_login()

💡 If you’re wondering why you don’t see any warnings about fine-tuning the model on a downstream task, that’s because for sequence-to-sequence tasks we keep all the weights of the network. Compare this to our text classification model in Chapter 3, where the head of the pretrained model was replaced with a randomly initialized network.

In [22]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors='tf')

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

In [24]:
tokenized_datasets = tokenized_datasets.remove_columns(
    english_dataset['train'].column_names
)

In [25]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11490
    })
})

Data collator is producing 4 outputs
- 1 for input ids
- 1 for attention mask
- 1 for labels
- 1 for decoder input ids (this are the labels only with right offset of 1)

In [26]:
features = [tokenized_datasets['train'][i] for i in range(2)]
# print(features,end="__")
data_collator(features)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[ 24728, 177913,    261, ...,   5176,    295,      1],
       [ 28544,    277,    263, ...,    259,   6661,      1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'labels': <tf.Tensor: shape=(2, 30), dtype=int32, numpy=
array([[ 22930,  51258,   4527,   7330,   8120, 110375,    265,   1689,
           263,   5883,   1117,    511,    259, 124624,    527,    790,
          8276,    263,    812,  15914,    259,    260,  19267,  38519,
           259,   6661,    790,   1070,    375,      1],
       [ 82085,    484,    259,   8174,    281,  63038,    281,  29804,
           418,   6956,    285,    351,    287,    313,   1565,    318,
         29280,  17070,    311,  35831,   1017,  51257,    764,   1398,
           823,    259,   6661,   2250,    418,      1]], dtype=int32)>, 'decoder_input_ids': <tf.Tensor: 

We’re almost ready to train! We just need to convert our datasets to tf.data.Datasets using the data collator we defined above, and then compile() and fit() the model. First, the datasets:

In [27]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets['train'],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)

tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets['validation'],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,
)

In [33]:
from transformers import create_optimizer
import tensorflow as tf

# the numberr of training steps is the number of samples in the datset, divided by the batched size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples//batch_size

num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split('/')[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy('mixed_float16')


In [34]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir = f'{model_name}-finetuned-mt5', tokenizer=tokenizer
)

model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset,callbacks=[callback],epochs=3
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/mt5-small-finetuned-mt5 is already a clone of https://huggingface.co/mukulsn/mt5-small-finetuned-mt5. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/3
1250/1250 [==============================] - ETA: 0s - loss: 5.3933

Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


1250/1250 [==============================] - 895s 702ms/step - loss: 5.3933 - val_loss: 2.8174
Epoch 2/3
1250/1250 [==============================] - 1020s 816ms/step - loss: 3.9908 - val_loss: 2.7397
Epoch 3/3
1250/1250 [==============================] - 1033s 826ms/step - loss: 3.7296 - val_loss: 2.7320


saving the model

In [35]:
model.save_pretrained("mt5-small", from_pt=True)

In [36]:
# mount it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
!cp -a /content/mt5-small-finetuned-mt5/. "/content/drive/MyDrive/Colab Notebooks/ML projects/Tensorflow projects/mt5 model/"

In [37]:
/content/drive/MyDrive/Colab Notebooks/ML projects/Tensorflow projects/

'/content'

In [45]:
from tqdm import tqdm
import numpy as np

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors='tf', pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets['validation'],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)

@tf.function(jit_compile=True)
def generate_with_xla(batch):
  return model.generate(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      max_new_tokens=32,
)

all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
  predictions=generate_with_xla(batch)
  decoded_preds=tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = labels.numpy()
  labels = np.where(labels!= -100, labels, tokenizer.pad_token_id)
  decoded_labels=tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_preds=['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  decoded_labels=['\n'.join(sent_tokenize(label.strip())) for label in decoded_labels]
  all_preds.extend(decoded_preds)
  all_labels.extend(decoded_labels)


100%|██████████| 1000/1000 [04:53<00:00,  3.41it/s]


In [46]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [49]:
from transformers import pipeline

hub_model_id = "mukulsn/mt5-small-finetuned-mt5"
summarizer = pipeline('summarization',model=hub_model_id)

tf_model.h5:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All model checkpoint layers were used when initializing TFMT5ForConditionalGeneration.

All the layers of TFMT5ForConditionalGeneration were initialized from the model checkpoint at mukulsn/mt5-small-finetuned-mt5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMT5ForConditionalGeneration for predictions without further training.


generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

In [50]:
def print_sumary(idx):
  review = english_dataset['test'][idx]['article']
  title = english_dataset['test'][idx]['highlights']
  summary = summarizer(english_dataset['test'][idx]['article'])[0]['summary_text']
  print(f"'>>> Article : {review}'")
  print(f"'>>> Highlight : {title}")
  print(f"'>>> Summary : {summary}")


In [52]:
print_sumary(100)

/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'>>> Article : (CNN)Anthony Ray Hinton is thankful to be free after nearly 30 years on Alabama's death row for murders he says he didn't commit. And incredulous that it took so long. Hinton, 58, looked up, took in the sunshine and thanked God and his lawyers Friday morning outside the county jail in Birmingham, minutes after taking his first steps as a free man since 1985. He spoke of unjustly losing three decades of his life, under fear of execution, for something he didn't do. "All they had to do was to test the gun, but when you think you're high and mighty and you're above the law, you don't have to answer to nobody," Hinton told reporters. "But I've got news for you -- everybody that played a part in sending me to death row, you will answer to God." Jefferson County Circuit Court Judge Laura Petro had ordered Hinton released after granting the state's motion to dismiss charges against him. Hinton was convicted of murder in the 1985 deaths of two Birmingham-area, fast-food restaura